# ResNet50 Binary Classification with Transfer Learning

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

## Check if TensorFlow is using the GPU
We check if TensorFlow is built with CUDA and if the GPU is available.
We also execute a simple matrix operation to test if the GPU is being used.


In [5]:
print("GPUs available:", tf.config.list_physical_devices('GPU'))
print("Is TensorFlow using the GPU?", tf.test.is_built_with_cuda())

# check which device is used for computations
tf.debugging.set_log_device_placement(True)

# simple TensorFlow operation to test GPU usage
a = tf.constant([[1.0, 2.0, 3.0]])
b = tf.constant([[4.0, 5.0, 6.0]])
c = tf.matmul(a, b, transpose_b=True)
print(c)

GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is TensorFlow using the GPU? True
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor([[32.]], shape=(1, 1), dtype=float32)


## Enable GPU memory growth
This prevents TensorFlow from allocating all GPU memory at once, which can lead to memory errors.

In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(e)


Physical devices cannot be modified after being initialized


We define paths for training and testing data:

In [7]:
train_dir = "../data/Training"
test_dir = "../data/Test"

We set image parameters based on the ResNet50 model, which needs images of size 224x224 pixels.

In [8]:
img_height = 224
img_width = 224
batch_size = 32

To pick the batch size, we need to consider the available GPU memory.
```bash
watch -n 1 nvidia-smi
```


We load the images from the directories considering they have this structure:
```
/Training/
    /Fire/
        image1.jpg
        image2.jpg
    /No_Fire/
        image1.jpg
        image2.jpg
/Test/
    /Fire/
        image1.jpg
        image2.jpg
    /No_Fire/
        image1.jpg
        image2.jpg
```

We use the ImageDataGenerator to load and preprocess images as they are already on folders:
- ImageDataGenerator works for any dataset, not just FLAME.
- It automatically loads, labels, resizes, and preprocesses images.
- It feeds data in batches, making training faster & memory efficient.
- Without it, you must manually load and preprocess every image.

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # (0 = No_Fire, 1 = Fire)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

## Load the model
We load ResNet50 with pretrained ImageNet weights, keeping only the feature extraction layers.


In [11]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # freeze base layers initially

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op FloorMod in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:loca

In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Convert feature maps into a single vector
x = Dropout(0.5)(x)  # Helps prevent overfitting
output_layer = Dense(1, activation='sigmoid')(x)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
ignVariableOp: (AssignVariableOp): /job:localhost/replica:0/task:0/device:GPU:0
resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
value: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
AssignVariableOp: (AssignVariableOp): /job:localhost/replica:0/task:0/device:GPU:0
resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
value: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
AssignVariableOp: (AssignVariableOp): /job:localhost/replica:0/task:0/device:GPU:0
resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
value: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
AssignVariableOp: (AssignVariableOp): /job:localhost/replica:0/task:0/device:GPU:0
resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
value: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
AssignVariableOp: (AssignV

2025-02-11 18:01:41.982503: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-11 18:01:41.982544: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-11 18:01:42.002403: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-11 18:01:42.002434: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-11 18:01:42.008866: I tensorflow/core/common_runtime/placer.cc:162] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-11 18:01:42.008896: I tensorflow/core/common_runtime/placer.cc:162] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
